# text mining

You may conduct these exercises using just the first first 20,000 reviews from the corpus. Use the first half for training and the rest for testing. Process reviews without capitalization or punctuation (and without using stemming or removing stopwords).

Q1. Build a sentiment analysis model that estimates star ratings from a 1,000 word bag-of-words model (based on the most popular words). 

Compare models based on:
(a) the 1,000 most common unigrams;
(b) the 1,000 most common bigrams;
(c) a model which uses a combination of unigrams and bigrams (i.e., some bigrams will be included if they are more popular than some unigrams, but the model dimensionality will still be 1,000). 

You may use a Ridge regression model (sklearn.linear model.Ridge) with a regularization coefficient of λ = 1). Report the MSE on the test set for each of the three variants, along with the five most positive and most negative tokens for each variant.

In [1]:
import os
os.chdir('/Users/shiqi/Downloads/cse258/week 5-6 Text Mining/')

from collections import defaultdict
import string
from sklearn import linear_model
import math
# using just the first first 20,000 reviews
def parseData(fname):
    for l in open(fname):
        yield eval(l)
data = list(parseData("goodreads_reviews_comics_graphic.json"))[:20000]

wordCount = defaultdict(int)
for d in data:
    for w in d['review_text'].split():
        wordCount[w] += 1

len(wordCount)

113928

In [2]:
# Process reviews without capitalization or punctuation
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

len(wordCount)

59814

In [3]:
# Just build our feature vector by taking the most popular words (lowercase, punctuation removed)
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
  r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
  for w in r.split():
    wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [4]:
# 1,000 word bag-of-words model (based on the most popular words)
words = [x[1] for x in counts[:1000]]

In [6]:
# the 1,000 most common unigrams
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review_text'].lower() if not c in punctuation])
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) # offset
    return feat

y = [d['rating'] for d in data]
y_train = y[:10000]
y_test = y[10000:]

X = [feature(d) for d in data]
X_train = X[:10000]
X_test = X[10000:]
# Regularized regression
clf = linear_model.Ridge(1.0, fit_intercept=False) # MSE + 1.0 l2
clf.fit(X_train, y_train)
theta = clf.coef_
uni_gram_pred = clf.predict(X_test)

In [7]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

print('unigram MSE = ',MSE(uni_gram_pred, y_test))

unigram MSE =  1.2390553477075856


In [10]:
wordSort = list(zip(theta[:-1], words))
wordSort.sort()
# Some of the most negative n-grams...
wordSort[:5]

[(-0.5297694170496591, 'boring'),
 (-0.5187275432483344, 'disappointing'),
 (-0.3449045061687771, 'says'),
 (-0.34445735816736345, 'worst'),
 (-0.3300903719725661, 'basically')]

In [9]:
# some of the most positive n-grams...
wordSort[-5:]

[(0.32142847836888355, '5'),
 (0.3429560361747083, 'yourself'),
 (0.370370980652106, 'beautifully'),
 (0.40761653981863377, 'mix'),
 (0.410004152269727, 'wait')]

In [11]:
# the 1,000 most common bigrams
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    ws = r.split()
    ws2 = [' '.join(x) for x in list(zip(ws[:-1],ws[1:]))]
    for w in ws2:
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review_text'].lower() if not c in punctuation])
    ws = r.split()
    ws2 = [' '.join(x) for x in list(zip(ws[:-1],ws[1:]))]
    for w in ws2:
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in data]
X_train = X[:10000]
X_test = X[10000:]

clf = linear_model.Ridge(1.0, fit_intercept=False) # MSE + 1.0 l2
clf.fit(X_train, y_train)
theta = clf.coef_
bi_gram_pred = clf.predict(X_test)

In [12]:
print('bigram MSE = ',MSE(bi_gram_pred, y_test))

bigram MSE =  1.2930626118603752


In [13]:
wordSort = list(zip(theta[:-1], words))
wordSort.sort()
# Some of the most negative bigrams...
wordSort[:5]

[(-1.0181651480170788, 'tuned for'),
 (-0.6712583885760911, 'miss your'),
 (-0.6208746381941644, 'the worst'),
 (-0.5485595567492433, 'a bad'),
 (-0.5396115156910215, 'too many')]

In [14]:
# Some of the most positive bigrams...
wordSort[-5:]

[(0.5647727696125389, 'reviews as'),
 (0.5670113632668727, '5 stars'),
 (0.6277280701070215, 'stay tuned'),
 (0.6362888029933468, 'cant wait'),
 (0.7930527784347808, 'forget to')]

In [15]:
# the 1,000 most common n_grams
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    ws = r.split()
    ws2 = [' '.join(x) for x in list(zip(ws[:-1],ws[1:]))]
    for w in ws + ws2:
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review_text'].lower() if not c in punctuation])
    ws = r.split()
    ws2 = [' '.join(x) for x in list(zip(ws[:-1],ws[1:]))]
    for w in ws + ws2:
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in data]
X_train = X[:10000]
X_test = X[10000:]

clf = linear_model.Ridge(1.0, fit_intercept=False) # MSE + 1.0 l2
clf.fit(X_train, y_train)
theta = clf.coef_
n_gram_pred = clf.predict(X_test)

In [16]:
# ngrams: a combination of unigrams and bigrams
print('ngrams MSE = ',MSE(n_gram_pred, y_test))

ngrams MSE =  1.2366939869514855


In [17]:
wordSort = list(zip(theta[:-1], words))
wordSort.sort()
# Some of the most negative n-grams...
wordSort[:5]

[(-0.38054599248630927, 'katies corner'),
 (-0.3662044311971771, 'share'),
 (-0.34264460246037415, 'what is'),
 (-0.32910038400902614, 'least'),
 (-0.28446911724166346, 'able to')]

In [18]:
# Some of the most positive n-grams...
wordSort[-5:]

[(0.31228135202807056, 'at least'),
 (0.35761224878132114, 'excellent'),
 (0.3695932391755788, 'wait'),
 (0.4933603984754807, 'able'),
 (0.5044389422876421, 'katies')]

Q2. Which review has the highest cosine similarity compared to the first review in the dataset, in terms of their tf-idf representations (using only the training set, and considering unigrams only)? Provide the ID and text of the review (2 marks).

In [19]:
# Q2 TF_IDFs
# using only the training set, and considering unigrams only
train = data[:10000]
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in train:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]

# Document frequency (df)
df = defaultdict(int)
for d in train:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in set(r.split()):
        df[w] += 1

In [20]:
# Term frequency (tf)
# Here we extract frequencies for terms in a single specific review
# the first review in the dataset
rev = train[0] # Query review
rev

tf = defaultdict(int)
r = ''.join([c for c in rev['review_text'].lower() if not c in punctuation])
for w in r.split():
    # Note = rather than +=, different versions of tf could be used instead
    tf[w] = 1
    
tfidf = dict(zip(words,[tf[w] * math.log2(len(train) / df[w]) for w in words]))
tfidfQuery = [tf[w] * math.log2(len(train) / df[w]) for w in words]

In [21]:
# Find the highest tf-idf words in our example review
maxTf = [(tf[w],w) for w in words]
maxTf.sort(reverse=True)
maxTfIdf = [(tfidf[w],w) for w in words]
maxTfIdf.sort(reverse=True)

maxTfIdf[:10]

[(7.356975041986563, 'vampire'),
 (6.930160374931366, 'crime'),
 (6.748553568441418, 'popular'),
 (6.687799537362322, 'nature'),
 (6.65835575946984, 'horrible'),
 (6.546245393148302, 'create'),
 (6.5328248773859805, 'date'),
 (6.50635266602479, 'strength'),
 (6.50635266602479, 'created'),
 (6.368849142274855, 'wouldnt')]

In [22]:
# Cosine similarity
def Cosine(x1,x2):
    numer = 0
    norm1 = 0
    norm2 = 0
    for a1,a2 in zip(x1,x2):
        numer += a1*a2
        norm1 += a1**2
        norm2 += a2**2
    if norm1*norm2:
        return numer / math.sqrt(norm1*norm2)
    return 0

# Find the other reviews in the corpus with the highest cosine similarity between tf-idf vectors
similarities = []
for rev2 in train:
    tf = defaultdict(int)
    r = ''.join([c for c in rev2['review_text'].lower() if not c in punctuation])
    for w in r.split():
        # Note = rather than +=
        tf[w] = 1
    tfidf2 = [tf[w] * math.log2(len(train) / df[w]) for w in words]
    similarities.append((Cosine(tfidfQuery, tfidf2), rev2['review_id'], rev2['review_text']))

In [23]:
similarities.sort(reverse=True)
similarities[:1]

[(1.0,
  '66b2ba840f9bd36d6d27f46136fe4772',
  'Sherlock Holmes and the Vampires of London \n Release Date: April 2014 \n Publisher: Darkhorse Comics \n Story by: Sylvain Cordurie \n Art by: Laci \n Colors by: Axel Gonzabo \n Cover by: Jean Sebastien Rossbach \n ISDN: 9781616552664 \n MSRP: $17.99 Hardcover \n "Sherlock Holmes died fighting Professor Moriarty in the Reichenbach Falls. \n At least, that\'s what the press claims. \n However, Holmes is alive and well and taking advantage of his presumed death to travel the globe. \n Unfortunately, Holmes\'s plans are thwarted when a plague of vampirism haunts Britain. \n This book collects Sherlock Holmes and the Vampires of London Volumes 1 and 2, originally created by French publisher Soleil." - Darkhorse Comics \n When I received this copy of "Sherlock Holmes and the Vampires of London" I was Ecstatic! The cover art was awesome and it was about two of my favorite things, Sherlock Holmes and Vampires. I couldn\'t wait to dive into this!

# Content, Structure, and Sequences

For these tasks, consider the complete dataset, but discard any users who have fewer than three interactions. Build a training set by considering all but the last interaction from each user (in order of timestamp). The test set will then contain each user’s most recent interaction.

Q3.Using the fastFM library1 compare three model variants to predict ratings: (adapting the FPMC Tensorflow code from Chapter 7, if using the Tensorflow code you can report and estimate the AUC by measuring how often the ‘predict’ function returns a positive score (positive item has a higher score than the negative item).)

(a) A regular latent-factor model, i.e., one which includes a user term and an item term (f(u,i));
(b) A non-personalized Markov Chain model, i.e., one which combines a one-hot encoding of the previous item with a one-hot encoding of the current item (f(i,i(prev))).
(c) A model which includes both the user, the item, and the previous item (f(u,i,i(prev))).

Report the training and test MSE for each variant; you may follow the same hyperparameter settings as in the starter code (3 marks).

I used  the Tensorflow code implements a Bayesian Personalized Ranking-style model

In [24]:
import dateutil
from collections import defaultdict
import tensorflow as tf
import random

In [25]:
def parseData(fname):
    for l in open(fname):
        yield eval(l)

userIDs = {}
itemIDs = {}
interactions = []
interactionsPerUser = defaultdict(list)
itemsPerUser = defaultdict(list)

for d in parseData("goodreads_reviews_comics_graphic.json"):
    u = d['user_id']
    i = d['book_id']
    t = d['date_added']
    r = d['rating']
    dt = dateutil.parser.parse(t)
    t = int(dt.timestamp())
    itemsPerUser[u].append((i,t,r))

In [26]:
user_id = itemsPerUser.keys()
for u in user_id:
    if len(itemsPerUser[u]) >= 3: # discard any users who have fewer than three interactions
        itemsPerUser[u][0][0]
        for d in range(len(itemsPerUser[u])):
            i = itemsPerUser[u][d][0]
            t = itemsPerUser[u][d][1]
            r = itemsPerUser[u][d][2]
            if not u in userIDs: userIDs[u] = len(userIDs)
            if not i in itemIDs: itemIDs[i] = len(itemIDs)
            interactions.append((t,u,i,r))
            interactionsPerUser[u].append((t,i,r))

In [27]:
# Interaction with timestamp
interactions[0]

(1470834408, 'bafc2d50014200cda7cb2b6acd60cd73', '6315584', 4)

In [28]:
len(interactions)

498312

In [29]:
interactions.sort()
itemIDs['dummy'] = len(itemIDs)
# Build a data structure including users, items, and their previous items
interactionsWithPrevious = []

for u in interactionsPerUser:
    interactionsPerUser[u].sort()
    lastItem = 'dummy'
    for (t,i,r) in interactionsPerUser[u]:
        interactionsWithPrevious.append((t,u,i,lastItem,r))
        lastItem = i

itemsPerUser = defaultdict(set)
for _,u,i,_ in interactions:
    itemsPerUser[u].add(i)
    
items = list(itemIDs.keys())

In [30]:
# Split the train and test
# sort by user and time
interactionsWithPrevious = sorted(interactionsWithPrevious, key=lambda element: (element[1], element[0]))
interactionsTrain = []
interactionsTest = []
# take the last interaction of each user as the test
for n in range(1,len(interactionsWithPrevious)):
    if interactionsWithPrevious[n][3] == 'dummy':
        test = interactionsWithPrevious[n-1]
        interactionsTest.append(test)
    else:
        train = interactionsWithPrevious[n-1]
        if train[3] != 'dummy':
            interactionsTrain.append(train)

In [31]:
# Define the tensorflow model. 

optimizer = tf.keras.optimizers.Adam(0.1)

class FPMC(tf.keras.Model):
    def __init__(self, K, lamb, UI = 1, IJ = 1):
        super(FPMC, self).__init__()
        # Initialize variables
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaUI = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaIU = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        self.gammaIJ = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        self.gammaJI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        # Regularization coefficient
        self.lamb = lamb
        # Which terms to include
        self.UI = UI
        self.IJ = IJ

    # Prediction for a single instance
    def predict(self, u, i, j):
        p = self.betaI[i] + self.UI * tf.tensordot(self.gammaUI[u], self.gammaIU[i], 1) +\
                            self.IJ * tf.tensordot(self.gammaIJ[i], self.gammaJI[j], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.nn.l2_loss(self.betaI) +\
                            tf.nn.l2_loss(self.gammaUI) +\
                            tf.nn.l2_loss(self.gammaIU) +\
                            tf.nn.l2_loss(self.gammaIJ) +\
                            tf.nn.l2_loss(self.gammaJI))

    def call(self, sampleU, # user
                   sampleI, # item
                   sampleJ, # previous item
                   sampleK): # negative item
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        j = tf.convert_to_tensor(sampleJ, dtype=tf.int32)
        k = tf.convert_to_tensor(sampleK, dtype=tf.int32)
        gamma_ui = tf.nn.embedding_lookup(self.gammaUI, u)
        gamma_iu = tf.nn.embedding_lookup(self.gammaIU, i)
        gamma_ij = tf.nn.embedding_lookup(self.gammaIJ, i)
        gamma_ji = tf.nn.embedding_lookup(self.gammaJI, j)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        x_uij = beta_i + self.UI * tf.reduce_sum(tf.multiply(gamma_ui, gamma_iu), 1) +\
                         self.IJ * tf.reduce_sum(tf.multiply(gamma_ij, gamma_ji), 1)
        gamma_uk = tf.nn.embedding_lookup(self.gammaUI, u)
        gamma_ku = tf.nn.embedding_lookup(self.gammaIU, k)
        gamma_kj = tf.nn.embedding_lookup(self.gammaIJ, k)
        gamma_jk = tf.nn.embedding_lookup(self.gammaJI, j)
        beta_k = tf.nn.embedding_lookup(self.betaI, k)
        x_ukj = beta_k + self.UI * tf.reduce_sum(tf.multiply(gamma_uk, gamma_ku), 1) +\
                         self.IJ * tf.reduce_sum(tf.multiply(gamma_kj, gamma_jk), 1)
        return -tf.reduce_mean(tf.math.log(tf.math.sigmoid(x_uij - x_ukj)))



def trainingStep(model, interactions):
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleJ, sampleK = [], [], [], []
        for _ in range(100000):
            _,u,i,j,_ = random.choice(interactions) # positive sample
            k = random.choice(items) # negative sample
            while k in itemsPerUser[u]:
                k = random.choice(items)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleJ.append(itemIDs[j])
            sampleK.append(itemIDs[k])

        loss = model(sampleU,sampleI,sampleJ,sampleK)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()


modelFPMC = FPMC(5, 0.001, 1, 1) # user, item, previous item
modelMF = FPMC(5, 0.001, 1, 0) # user, item
modelFMC = FPMC(5, 0.001, 0, 1) # item, previous item


interactionsTestPerUser = defaultdict(set)
itemSet = set()
for _,u,i,j,_ in interactionsTest:
    interactionsTestPerUser[u].add((i,j))
    itemSet.add(i)
    itemSet.add(j)

def AUCu(model, u, N):
    win = 0
    if N > len(interactionsTestPerUser[u]):
        N = len(interactionsTestPerUser[u])
    positive = random.sample(interactionsTestPerUser[u],N)
    negative = random.sample(itemSet,N)
    for (i,j),k in zip(positive,negative):
        sp = model.predict(userIDs[u], itemIDs[i], itemIDs[j]).numpy()
        sn = model.predict(userIDs[u], itemIDs[k], itemIDs[j]).numpy()
        if sp > sn:
            win += 1
    return win/N

def AUC(model):
    av = []
    for u in interactionsTestPerUser:
        av.append(AUCu(model, u, 10))
    return sum(av) / len(av)

In [32]:
for model,name in [(modelFPMC,"FPMC"), (modelMF,"MF"), (modelFMC,"FMC")]:
    for i in range(100):
        obj = trainingStep(model, interactionsTrain)
        if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))
    print(name + " AUC = " + str(AUC(model)))


iteration 10, objective = 0.8851714
iteration 20, objective = 0.79832035
iteration 30, objective = 0.6918548
iteration 40, objective = 0.6882886
iteration 50, objective = 0.6807407
iteration 60, objective = 0.6786849
iteration 70, objective = 0.6778456
iteration 80, objective = 0.6770646
iteration 90, objective = 0.67785627
iteration 100, objective = 0.6778191
FPMC AUC = 0.7089346655153982
iteration 10, objective = 1.117121
iteration 20, objective = 0.90041363
iteration 30, objective = 0.7413565
iteration 40, objective = 0.6986629
iteration 50, objective = 0.687025
iteration 60, objective = 0.68199456
iteration 70, objective = 0.68072826
iteration 80, objective = 0.6794847
iteration 90, objective = 0.6786478
iteration 100, objective = 0.6791993
MF AUC = 0.7016981209654208
iteration 10, objective = 1.1579998
iteration 20, objective = 0.9019433
iteration 30, objective = 0.8522896
iteration 40, objective = 0.75185883
iteration 50, objective = 0.69575876
iteration 60, objective = 0.6846295

From the AUC results, we can tell that the FPMC > FMC > MF model. The FPMC model works best for the binary prediction of items.

Q4. Experiment with the factorization machine code to see whether performance of the above models can be improved by incorporating any other features from the data (1 mark).4 If you can’t get any FM library working, you may skip this question and simply describe what features you might try.

I might try to:

1) include text content to predict ratings.
From Taks 1, I already get some most poitive and negative unigrams and bigrams. I can assign each review_text with a positive ranking and predict the ratings.

2) include previous ratings.
Previous rating might have prediction power in terms of evaluating the user's rating standard. Some users in general tend to give higher/lower ratings than other.